# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Modify-ecoinvent" data-toc-modified-id="Modify-ecoinvent-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modify ecoinvent</a></div><div class="lev1 toc-item"><a href="#Prepare-additional-datasets" data-toc-modified-id="Prepare-additional-datasets-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare additional datasets</a></div><div class="lev2 toc-item"><a href="#import-from-excel" data-toc-modified-id="import-from-excel-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>import from excel</a></div><div class="lev1 toc-item"><a href="#Define-REMIND-scenarios-and-years-to-be-used:" data-toc-modified-id="Define-REMIND-scenarios-and-years-to-be-used:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define REMIND scenarios and years to be used:</a></div><div class="lev1 toc-item"><a href="#Modify-all-electricity-generation-and-market-datasets:" data-toc-modified-id="Modify-all-electricity-generation-and-market-datasets:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modify all electricity generation and market datasets:</a></div><div class="lev1 toc-item"><a href="#create-current-version-of-ecoinvent-with-additional-technologies-added-in:" data-toc-modified-id="create-current-version-of-ecoinvent-with-additional-technologies-added-in:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>create current version of ecoinvent with additional technologies added in:</a></div>

# Modify ecoinvent

In this notebook we actually run the functions that we've been creating so that we can modify ecoinvent!

We create 3 new versions of ecoinvent.

1. ecoinvent 2012, which has updates based on image results from 2012
2. ecoinvent 2040 BAU, which is a 2040 version of ecoinvent based on the image BAU scenario
3. ecoinvent 2040 450, which is a 2040 version of ecoinvent based on the image 450 ppm scenario


In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [ ]:
%matplotlib inline

Import useful libraries and set project:

In [ ]:
%run initialize_notebook.ipynb

Import functions that are written on top of Wurst:

In [ ]:
import helpers.eimod as eimod
import helpers.ei2rmnd as ei2rmnd

# Prepare additional datasets

We import some additional datasets for carbon capture and storage electricity. These have been prepared in Simapro and exported. We have cleaned up the datasets and stored them in excel using Brightway input output functionality. We read them from excel here:

## import from excel

In [ ]:
import_karma()

# Define REMIND scenarios and years to be used:

Here we set the scenarios and years for which we want to use when creating a new version of ecoinvent:

In [ ]:
database_dict = collections.defaultdict(dict)

for year in [2015, 2030, 2050]:
#for year in [2015]:
    for scenario in ['BAU','RCP26','RCP37']:
        if year == 2015 and scenario != 'BAU':
            continue
        db_name = 'ecoinvent_Remind_' + scenario + '_' + str(year)
        database_dict[db_name] = {'year' : year , 'scenario' : scenario}

#database_dict['test'] = {'year' : 2041 , 'scenario' : 'BAU'}

In [ ]:
database_dict

In [ ]:
databases

# Modify all electricity generation and market datasets:

This cell uses the Wurst functionality as well as the functions that we wrote in the notebook "_Functions_to_modify_ecoinvent.ipynb" to make the changes to ecoinvent:

In [ ]:
for key in pyprind.prog_bar(database_dict.keys()):
    
    db = extract_brightway2_databases(['Carma CCS','ecoinvent_3.5'])
    default_global_location(db)
    eimod.fix_unset_technosphere_and_production_exchange_locations(db)
    #set_global_location_for_additional_datasets(db)
    eimod.remove_nones(db)
    eimod.rename_locations(db, eimod.fix_names)
    #add_negative_CO2_flows_for_biomass_CCS(db)
    
    year = database_dict[key]['year']
    scenario = database_dict[key]['scenario']
    remind_data = eimod.get_remind_data(scenario)
    print(key)
       
    #Electricity generation datasets:
    print('Changing ecoinvent electricity generation datasets')
    technology_changes = eimod.update_electricity_datasets_with_remind_data(db, remind_data, year, agg_func=np.average, update_efficiency = True, update_emissions = True)
    
    # Electricity markets:
    print('Changing electricity Markets')
    market_changes = eimod.update_electricity_markets(db, year, remind_data)
    
    #Electricity generation datasets from project Carma
    print('Changing Carma electricity datasets')
    eimod.modify_all_carma_electricity_datasets(db, remind_data, year, update_efficiency = True, update_emissions = True)
    
    print('Saving changes to excel')
    tech_df = pd.DataFrame.from_dict(technology_changes)
    tech_df.index = pd.MultiIndex.from_tuples(tech_df.index)
    tech_df=tech_df.T
    tech_df = tech_df.set_index([('meta data','remind technology'),('meta data','name'),('meta data','location')], drop=True).sort_index()
    market_df = pd.DataFrame.from_dict(market_changes).T
    market_df = market_df.set_index([('meta data','name'),('meta data','location')], drop=True).sort_index().T

    writer = pd.ExcelWriter('electricity changes '+ str(year) + ' '+ scenario + '.xlsx')
    market_df.to_excel(writer, sheet_name = 'markets')
    for tech in tech_df.index.levels[0]:
        tech_df.loc[tech].dropna(how='all', axis=1).swaplevel(i=0,j=1, axis=1).T.to_excel(writer, sheet_name = tech)
    writer.save()
    del tech_df
    del market_df
    del writer
    
    
    eimod.rename_locations(db, eimod.fix_names_back)
    if key in databases:
        del databases[key]

    write_brightway2_database(db, key)

    lca = LCA({Database(key).random(): 1}, method=lcia_methods['CC'])
    lca.lci()
    lca.lcia()
    print("Test LCA Score: {}".format(lca.score))


# create current version of ecoinvent with additional technologies added in:

In [ ]:
key = 'ecoinvent_added_technologies'
    
db = extract_brightway2_databases(['Carma CCS','ecoinvent_3.5'])
default_global_location(db)
fix_unset_technosphere_and_production_exchange_locations(db)
#set_global_location_for_additional_datasets(db)
remove_nones(db)
rename_locations(db, fix_names)
add_negative_CO2_flows_for_biomass_CCS(db)
    
rename_locations(db, fix_names_back)
if key in databases:
    del databases[key]

write_brightway2_database(db, key)
del db
    
        
lca = LCA({Database(key).random(): 1}, method=lcia_methods['CC'])
lca.lci()
lca.lcia()
print(lca.score)